In [1]:
import os
import torch
from vllm import LLM, SamplingParams
from datasets import load_dataset
from collections import defaultdict
from sympy import simplify, latex
from latex2sympy2 import latex2sympy

/root/miniconda/envs/sal/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-02 18:18:02,135	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/root/miniconda/envs/sal/lib/python3.11/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION


In [2]:
config = {
    "model_path": "meta-llama/Llama-3.1-8B-Instruct",
    "dataset_name": "HuggingFaceH4/MATH-500",
    "dataset_split": "test",
    "prompt": """Solve the following math problem efficiently and clearly:

- For simple problems (2 steps or fewer):
Provide a concise solution with minimal explanation.

- For complex problems (3 steps or more):
Use this step-by-step format:

## Step 1: [Concise description]
[Brief explanation and calculations]

## Step 2: [Concise description]
[Brief explanation and calculations]

...

Therefore, the final answer is: $\\boxed{answer}$. I hope it is correct.

Where [answer] is just the final number or expression that solves the problem.""",
    "num_completions": 3,
    "temperature": 0.8,
    "top_p": 1.0,
    "max_tokens": 2048
}

In [3]:
sampling_params = SamplingParams(
    temperature=config["temperature"],
    top_p=config["top_p"],
    max_tokens=config["max_tokens"],
    n=config["num_completions"],
)
llm = LLM(model=config["model_path"])


WARNING 08-02 18:18:14 arg_utils.py:953] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 08-02 18:18:14 config.py:1005] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 08-02 18:18:14 llm_engine.py:237] Initializing an LLM engine (vdev) with config: model='meta-llama/Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, qu

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  5.81it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:01,  1.94it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.56it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.55it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.68it/s]



INFO 08-02 18:18:20 model_runner.py:1071] Loading model weights took 14.9888 GB
INFO 08-02 18:18:21 gpu_executor.py:122] # GPU blocks: 28041, # CPU blocks: 2048
INFO 08-02 18:18:21 gpu_executor.py:126] Maximum concurrency for 131072 tokens per request: 3.42x
INFO 08-02 18:18:22 model_runner.py:1402] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 08-02 18:18:22 model_runner.py:1406] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 08-02 18:18:31 model_runner.py:1530] Graph capturing finished in 9 secs.


In [10]:
completions = llm.generate("what is the capital of South Korea?", sampling_params=sampling_params)
answers = [c.outputs[0].text for c in completions]

answers

Processed prompts: 100%|██████████| 1/1 [00:15<00:00, 15.26s/it, est. speed input: 0.59 toks/s, output: 402.55 toks/s]


[' \nSeoul is the capital of South Korea. Educated guess: you probably have already known this fact. Seoul is a megacity and the largest city in South Korea. It is the country\'s economic, cultural, and political center, with a diverse economy, vibrant culture, and rich history.\nDid you know that Seoul is known as the "City of Morning Calm"? It has a unique blend of modern and traditional architecture, with historic sites like Gyeongbokgung Palace and Bukchon Hanok Village, along with trendy districts like Hongdae and Gangnam.\nSo, if you ever find yourself in South Korea, Seoul is a must-visit destination!\n\nThe answer to your question is: **Seoul**. \nWould you like to know more about Seoul or South Korea? Feel free to ask! \nAlso, if you have any more questions or topics you\'d like to discuss, I\'m here to help! \nLet\'s connect and explore! \nHope this helps! Let me know if you have any other questions! \nAlso, feel free to ask me any other questions you may have. I\'m here to h

In [19]:
token_ids_list = [output.token_ids for c in completions for output in c.outputs]
len(token_ids_list[0])

2048

In [4]:
import time
n=64

sampling_params = SamplingParams(
    temperature=config["temperature"],
    top_p=config["top_p"],
    max_tokens=config["max_tokens"],
    n=64,
    seed=0
)
start = time.time()
completions = llm.generate("what is the capital of South Korea?", sampling_params=sampling_params)
elapsed = time.time() - start
print(f"n={n}, elapsed time: {elapsed:.2f} sec")

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:25<00:00, 25.41s/it, est. speed input: 0.35 toks/s, output: 5044.02 toks/s]

n=64, elapsed time: 25.42 sec


# Majority Voting

In [52]:
from collections import defaultdict
from datasets import load_dataset
from sympy import simplify
from sympy.parsing.latex import parse_latex as latex2sympy
from sympy.printing.latex import latex
from vllm import LLM, SamplingParams

# Canonical form 처리
def get_canonical_form(expr: str) -> str:
    try:
        return latex(simplify(latex2sympy(expr)))
    except Exception:
        return expr.strip()

# Majority voting
def find_majority_answer(answers: list) -> str:
    canonical_groups = defaultdict(int)
    canonical_to_original = {}

    for answer in answers:
        answer_text = getattr(answer, "text", str(answer))
        canonical = get_canonical_form(answer_text)
        canonical_groups[canonical] += 1

        if canonical not in canonical_to_original:
            canonical_to_original[canonical] = answer_text

    max_count = max(canonical_groups.values())
    for canonical, count in canonical_groups.items():
        if count == max_count:
            return canonical_to_original[canonical]

# 프롬프트 1개씩 majority vote 처리
def majority_vote(batch, config, sampling_params, llm):
    prompt = batch["problem"]
    full_prompt = f"{config['prompt'].strip()}\n\n### Problem:\n{prompt.strip()}"
    completions = llm.generate(full_prompt, sampling_params=sampling_params)
    pred = find_majority_answer(completions)

    return {
        "completions": [c.outputs[0].text for c in completions],
        "pred": pred,
    }


In [53]:
from datasets import Dataset, load_dataset

dataset = load_dataset(config['dataset_name'], split=config['dataset_split'])
dataset = dataset.select(range(2))


# Majority voting 실행 (prompt 하나씩)
dataset = dataset.map(
    majority_vote,
    batched=False,  # 개별 example에 대해 실행
    fn_kwargs={"config": config, "sampling_params": sampling_params, "llm": llm},
    desc="Running majority voting",
    load_from_cache_file=False,
)

# 결과 출력
for i in range(2):
    print(f"\n--- Problem #{i+1} ---")
    print("Prompt:", dataset[i]["problem"])
    print("Predicted Answer:", dataset[i]["pred"])
    print("Completions:", dataset[i]["completions"])

Running majority voting: 100%|██████████| 2/2 [00:48<00:00, 24.32s/ examples]


--- Problem #1 ---
Prompt: Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$
Predicted Answer: RequestOutput(request_id=6, prompt='Solve the following math problem efficiently and clearly:\n\n- For simple problems (2 steps or fewer):\nProvide a concise solution with minimal explanation.\n\n- For complex problems (3 steps or more):\nUse this step-by-step format:\n\n## Step 1: [Concise description]\n[Brief explanation and calculations]\n\n## Step 2: [Concise description]\n[Brief explanation and calculations]\n\n...\n\nTherefore, the final answer is: $\\boxed{answer}$. I hope it is correct.\n\nWhere [answer] is just the final number or expression that solves the problem.\n\n### Problem:\nConvert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\\theta),$ where $r > 0$ and $0 \\le \\theta < 2 \\pi.$', prompt_token_ids=[128000,

In [54]:
from sympy import Eq, simplify
from sympy.parsing.latex import parse_latex

def is_correct(pred_list, answer_list) -> bool:
    try:
        pred_str = pred_list[0] if isinstance(pred_list, list) else pred_list
        answer_str = answer_list[0] if isinstance(answer_list, list) else answer_list

        lhs = simplify(latex2sympy(pred_str))
        rhs = simplify(latex2sympy(answer_str))
        return lhs == rhs
    except Exception:
        return str(pred_str).strip() == str(answer_str).strip()

correct_count = 0
total = len(dataset)

for i in range(total):
    pred = dataset[i]["pred"]
    answer = dataset[i]["answer"]  # 정답 필드명이 다르면 바꾸세요

    if is_correct(pred, answer):
        correct_count += 1

accuracy = correct_count / total
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 0.00%


In [59]:
from collections import defaultdict
import re
from sympy import simplify
from sympy.parsing.latex import parse_latex

# Canonicalize helpers
def get_canonical_form(expr: str) -> str:
    try:
        return latex(simplify(latex2sympy(expr)))
    except Exception:
        return expr.strip()

def extract_boxed_answer(text: str) -> str:
    match = re.search(r"\\boxed\{(.+?)\}", text)
    if match:
        return match.group(1).strip()
    return text.strip()

# Majority voting logic
def find_majority_answer(answers: list) -> str:
    canonical_groups = defaultdict(int)
    canonical_to_original = {}

    print("\n--- Raw Completions ---")
    for i, answer in enumerate(answers):
        try:
            answer_text = answer.outputs[0].text
        except Exception:
            answer_text = str(answer)

        print(f"[{i}] {answer_text}")

        # ✨ 정답 박스만 추출해서 canonical 비교
        extracted = extract_boxed_answer(answer_text)
        canonical = get_canonical_form(extracted)
        canonical_groups[canonical] += 1

        if canonical not in canonical_to_original:
            canonical_to_original[canonical] = extracted  # ✨ 여기서도 정제된 형태 저장

    print("\n--- Canonical Groups ---")
    for canonical, count in canonical_groups.items():
        print(f"{canonical}: {count}")

    max_count = max(canonical_groups.values())
    for canonical, count in canonical_groups.items():
        if count == max_count:
            selected = canonical_to_original[canonical]
            print(f"\n>>> Selected: {selected}")
            return selected  # ✨ 정제된 answer만 반환

def is_correct(pred_str, answer_str) -> bool:
    try:
        lhs = simplify(parse_latex(pred_str))
        rhs = simplify(parse_latex(extract_boxed_answer(answer_str)))
        return lhs == rhs
    except Exception as e:
        print(f"[WARN] Failed to parse or simplify: {e}")
        return pred_str.strip() == answer_str.strip()

# Example here

In [69]:
sample = dataset[0]["problem"]
print('-'*100)
print(sample)

prompt = f"{config['prompt'].strip()}\n\n### Problem:\n{sample.strip()}"
print('-'*100)
print(prompt)

----------------------------------------------------------------------------------------------------
Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$
----------------------------------------------------------------------------------------------------
Solve the following math problem efficiently and clearly:

- For simple problems (2 steps or fewer):
Provide a concise solution with minimal explanation.

- For complex problems (3 steps or more):
Use this step-by-step format:

## Step 1: [Concise description]
[Brief explanation and calculations]

## Step 2: [Concise description]
[Brief explanation and calculations]

...

Therefore, the final answer is: $\boxed{answer}$. I hope it is correct.

Where [answer] is just the final number or expression that solves the problem.

### Problem:
Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer i

In [75]:
completions = llm.generate(prompt, sampling_params=sampling_params)

for i, c in enumerate(completions):
    print(f"\n=== Completion {i} ===")
    print(f"Request ID: {c.request_id}")
    print(f"Prompt:\n{c.prompt}")
    print("Outputs:")
    for j, o in enumerate(c.outputs):
        print(f"  [{j}] {o.text}")

Processed prompts: 100%|██████████| 1/1 [00:15<00:00, 15.20s/it, est. speed input: 10.92 toks/s, output: 134.71 toks/s]


=== Completion 0 ===
Request ID: 13
Prompt:
Solve the following math problem efficiently and clearly:

- For simple problems (2 steps or fewer):
Provide a concise solution with minimal explanation.

- For complex problems (3 steps or more):
Use this step-by-step format:

## Step 1: [Concise description]
[Brief explanation and calculations]

## Step 2: [Concise description]
[Brief explanation and calculations]

...

Therefore, the final answer is: $\boxed{answer}$. I hope it is correct.

Where [answer] is just the final number or expression that solves the problem.

### Problem:
Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$
Outputs:
  [0] 

## Step 1: To convert the point $(0,3)$ from rectangular coordinates to polar coordinates, we need to calculate the radius $r$ and the angle $\theta$ using the formulas $r = \sqrt{x^2 + y^2}$ and $\theta = \tan^{-1}\left(\frac{y}{x}\ri

In [120]:
import re

def extract_boxed_answer(text: str) -> str:
    """
    $\boxed{...}$ 안의 내용을 정확하게 추출 (중첩 괄호 지원)
    """
    # 중첩 괄호를 포함한 boxed 수식 전체 추출
    pattern = r"\\boxed\{((?:[^{}]|\{[^{}]*\})*)\}"
    match = re.search(pattern, text)
    if match:
        return match.group(1).strip()
    return text.strip()  # fallback

completion_text = completions[0].outputs[0].text
answer = extract_boxed_answer(completion_text).replace(" ", "")
print("🎯 Extracted Answer:", answer)

🎯 Extracted Answer: \left(3,\frac{\pi}{2}\right)


In [122]:
from typing import Dict, List

import datasets


def process_docs(dataset: datasets.Dataset) -> datasets.Dataset:
    def _process_doc(doc: dict) -> dict:
        out_doc = {
            "problem": doc["problem"],
            "solution": doc["solution"],
            "answer": remove_boxed(last_boxed_only_string(doc["solution"])),
        }
        return out_doc

    return dataset.map(_process_doc)


def process_results(doc: dict, results: List[str]) -> Dict[str, int]:
    retval = 0
    indices = [pos for pos, char in enumerate(results[0]) if char == "$"]
    if len(indices) <= 1:
        answer = results[0]
    else:
        answer = results[0][indices[0] + 1 : indices[-1]]

    if is_equiv(answer, remove_boxed(last_boxed_only_string(doc["solution"]))):
        retval = 1

    results = {
        "exact_match": retval,
    }
    return results


# string normalization from https://github.com/EleutherAI/lm-evaluation-harness/blob/master/lm_eval/tasks/hendrycks_math.py
def is_equiv(str1, str2, verbose=False):
    if str1 is None and str2 is None:
        print("WARNING: Both None")
        return True
    if str1 is None or str2 is None:
        return False

    try:
        ss1 = strip_string(str1)
        ss2 = strip_string(str2)
        if verbose:
            print(ss1, ss2)
        return ss1 == ss2
    except Exception:
        return str1 == str2


def remove_boxed(s):
    if "\\boxed " in s:
        left = "\\boxed "
        assert s[: len(left)] == left
        return s[len(left) :]

    left = "\\boxed{"

    assert s[: len(left)] == left
    assert s[-1] == "}"

    return s[len(left) : -1]


def last_boxed_only_string(string):
    idx = string.rfind("\\boxed")
    if "\\boxed " in string:
        return "\\boxed " + string.split("\\boxed ")[-1].split("$")[0]
    if idx < 0:
        idx = string.rfind("\\fbox")
        if idx < 0:
            return None

    i = idx
    right_brace_idx = None
    num_left_braces_open = 0
    while i < len(string):
        if string[i] == "{":
            num_left_braces_open += 1
        if string[i] == "}":
            num_left_braces_open -= 1
            if num_left_braces_open == 0:
                right_brace_idx = i
                break
        i += 1

    if right_brace_idx is None:
        retval = None
    else:
        retval = string[idx : right_brace_idx + 1]

    return retval


def fix_fracs(string):
    substrs = string.split("\\frac")
    new_str = substrs[0]
    if len(substrs) > 1:
        substrs = substrs[1:]
        for substr in substrs:
            new_str += "\\frac"
            if substr[0] == "{":
                new_str += substr
            else:
                try:
                    assert len(substr) >= 2
                except AssertionError:
                    return string
                a = substr[0]
                b = substr[1]
                if b != "{":
                    if len(substr) > 2:
                        post_substr = substr[2:]
                        new_str += "{" + a + "}{" + b + "}" + post_substr
                    else:
                        new_str += "{" + a + "}{" + b + "}"
                else:
                    if len(substr) > 2:
                        post_substr = substr[2:]
                        new_str += "{" + a + "}" + b + post_substr
                    else:
                        new_str += "{" + a + "}" + b
    string = new_str
    return string


def fix_a_slash_b(string):
    if len(string.split("/")) != 2:
        return string
    a = string.split("/")[0]
    b = string.split("/")[1]
    try:
        a = int(a)
        b = int(b)
        assert string == "{}/{}".format(a, b)
        new_string = "\\frac{" + str(a) + "}{" + str(b) + "}"
        return new_string
    except AssertionError:
        return string


def remove_right_units(string):
    # "\\text{ " only ever occurs (at least in the val set) when describing units
    if "\\text{ " in string:
        splits = string.split("\\text{ ")
        assert len(splits) == 2
        return splits[0]
    else:
        return string


def fix_sqrt(string):
    if "\\sqrt" not in string:
        return string
    splits = string.split("\\sqrt")
    new_string = splits[0]
    for split in splits[1:]:
        if split[0] != "{":
            a = split[0]
            new_substr = "\\sqrt{" + a + "}" + split[1:]
        else:
            new_substr = "\\sqrt" + split
        new_string += new_substr
    return new_string


def strip_string(string):
    # linebreaks
    string = string.replace("\n", "")

    # remove inverse spaces
    string = string.replace("\\!", "")

    # replace \\ with \
    string = string.replace("\\\\", "\\")

    # replace tfrac and dfrac with frac
    string = string.replace("tfrac", "frac")
    string = string.replace("dfrac", "frac")

    # remove \left and \right
    string = string.replace("\\left", "")
    string = string.replace("\\right", "")

    # Remove circ (degrees)
    string = string.replace("^{\\circ}", "")
    string = string.replace("^\\circ", "")

    # remove dollar signs
    string = string.replace("\\$", "")

    # remove units (on the right)
    string = remove_right_units(string)

    # remove percentage
    string = string.replace("\\%", "")
    string = string.replace("\%", "")  # noqa: W605

    # " 0." equivalent to " ." and "{0." equivalent to "{." Alternatively, add "0" if "." is the start of the string
    string = string.replace(" .", " 0.")
    string = string.replace("{.", "{0.")
    # if empty, return empty string
    if len(string) == 0:
        return string
    if string[0] == ".":
        string = "0" + string

    # to consider: get rid of e.g. "k = " or "q = " at beginning
    if len(string.split("=")) == 2:
        if len(string.split("=")[0]) <= 2:
            string = string.split("=")[1]

    # fix sqrt3 --> sqrt{3}
    string = fix_sqrt(string)

    # remove spaces
    string = string.replace(" ", "")

    # \frac1b or \frac12 --> \frac{1}{b} and \frac{1}{2}, etc. Even works with \frac1{72} (but not \frac{72}1). Also does a/b --> \\frac{a}{b}
    string = fix_fracs(string)

    # manually change 0.5 --> \frac{1}{2}
    if string == "0.5":
        string = "\\frac{1}{2}"

    # NOTE: X/Y changed to \frac{X}{Y} in dataset, but in simple cases fix in case the model output is X/Y
    string = fix_a_slash_b(string)

    return string


In [79]:
i = 0  # 첫 번째 예제 기준 (원하는 index로 변경 가능)
print("📌 문제:", dataset[i]["problem"])
print(dataset[i]["answer"])

📌 문제: Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$
✅ 정답 (GT): \left( 3, \frac{\pi}{2} \right)


In [85]:
print(dataset[i]["answer"])
print(answer)

\left( 3, \frac{\pi}{2} \right)
\left(3, \frac{\pi}{2}\right)


In [99]:
def normalize_latex(expr: str) -> str:
    # 공백 제거 + \left, \right 제거 + 괄호 제거
    expr = expr.strip()
    expr = re.sub(r"\\left|\\right", "", expr)
    expr = re.sub(r"\s+", "", expr)
    expr = expr.replace("(", "").replace(")", "")
    return expr

def is_correct(pred_str: str, answer_str: str) -> bool:
    norm_pred = normalize_latex(pred_str)
    norm_ans = normalize_latex(answer_str)
    return norm_pred == norm_ans

pred = r"\left(3, \frac{\pi}{2}\right)"
gt   = r"\left( 3, \frac{\pi}{2} \right)"

print(normalize_latex(pred))  # 3,\frac{\pi}{2}
print(normalize_latex(gt))    # 3,\frac{\pi}{2}
print(is_correct(pred, gt))   # ✅ True

3,\frac{\pi}{2}
3,\frac{\pi}{2}
True


In [111]:
from sympy.parsing.latex import parse_latex
from sympy import symbols

# 예제 1: 분수 표현식 파싱
try:
    expr = parse_latex(r"\frac{1 + \sqrt{a}}{b}", backend="lark")
    print("Parsed Expression:", expr)

    # 변수 값 대입 후 근사값 계산
    a, b = symbols("a b")
    result = expr.evalf(4, subs={a: 5, b: 2})
    print("Evaluated Result (a=5, b=2):", result)

except Exception as e:
    print("[Error] 예제 1 실패:", e)

print("\n" + "-"*40 + "\n")

# 예제 2: 적분 표현식 파싱
try:
    func = parse_latex(r"\int_1^\alpha \dfrac{\mathrm{d}t}{t}", backend="lark")
    print("Parsed Integral:", func)

    # alpha 대입 후 계산
    alpha = symbols("alpha")
    result2 = func.evalf(subs={alpha: 2})
    print("Evaluated Integral (alpha=2):", result2)

except Exception as e:
    print("[Error] 예제 2 실패:", e)

Parsed Expression: (sqrt(a) + 1)/b
Evaluated Result (a=5, b=2): 1.618

----------------------------------------

Parsed Integral: Integral(1/t, (t, 1, alpha))
Evaluated Integral (alpha=2): 0.693147180559945


In [119]:
from latex2sympy2 import latex2sympy
from sympy import latex, simplify

def get_canonical_form(expression: str) -> str:
    parsed_expr = latex2sympy(expression)
    simplified_expr = simplify(parsed_expr)
    return latex(simplified_expr)

pred = r"\left(3, \frac{\pi}{2}\right)"
gt   = r"\left( 3, \frac{\pi}{2} \right)"

def clean_latex_expr(latex: str) -> str:
    """SymPy가 파싱할 수 있도록 LaTeX 표현 정리"""
    return latex.replace(r"\left(", "").replace(r"\right)", "")

# 문제 예시
cleaned = clean_latex_expr(pred)

print("🧼 Cleaned LaTeX:", cleaned)

try:
    parsed = get_canonical_form(cleaned)
    print("✅ Parsed:", parsed)
except Exception as e:
    print("❌ Failed to parse:", e)



🧼 Cleaned LaTeX: 3, \frac{\pi}{2}
❌ Failed to parse: 'list' object has no attribute 'replace'


In [88]:
from sympy.parsing.latex import parse_latex

def is_correct(pred_str, answer_str) -> bool:
    try:
        lhs = simplify(parse_latex(pred_str))
        rhs = simplify(parse_latex(extract_boxed_answer(answer_str)))
        return lhs == rhs
    except Exception as e:
        print(f"[WARN] Failed to parse or simplify: {e}")
        return pred_str.strip() == answer_str.strip()


if is_correct(dataset[i]["answer"], answer):
    print('Correct')
else: print("False")

[WARN] Failed to parse or simplify: 'NoneType' object is not callable
False


In [61]:
sample = dataset[0]["problem"]
prompt = f"{config['prompt'].strip()}\n\n### Problem:\n{sample.strip()}"
completions = llm.generate(prompt, sampling_params=sampling_params)

pred = find_majority_answer(completions)
print(f"\nMajority Vote Prediction: {pred}")

print("\n🔍 Now scoring...")

correct = 0
total = len(dataset)

for i in range(total):
    pred = extract_boxed_answer(dataset[i]["pred"][0])  # 🔧 정제
    answer = dataset[i]["answer"]
    if is_correct(pred, answer):
        correct += 1

print(f"✅ Accuracy: {correct}/{total} = {correct / total:.2%}")

Processed prompts: 100%|██████████| 1/1 [00:23<00:00, 23.69s/it, est. speed input: 7.01 toks/s, output: 4048.86 toks/s]


--- Raw Completions ---
[0] 

## Step 1: Recall the conversion formulas
The conversion from rectangular to polar coordinates uses the formulas $r = \sqrt{x^2 + y^2}$ for the radial coordinate and $\theta = \text{atan2}(y,x)$ for the angular coordinate.

## Step 2: Identify the rectangular coordinates
The given point is $(x,y) = (0,3)$.

## Step 3: Calculate the radial coordinate
Substitute the given values into the formula for $r$: $r = \sqrt{0^2 + 3^2} = \sqrt{9} = 3$.

## Step 4: Calculate the angular coordinate
Substitute the given values into the formula for $\theta$: $\theta = \text{atan2}(3,0) = \frac{\pi}{2}$, since the point lies on the positive y-axis.

## Step 5: Write the polar coordinates
Combine the calculated values of $r$ and $\theta$ to obtain the polar coordinates: $(r,\theta) = \left(3, \frac{\pi}{2}\right)$.

The final answer is: $\boxed{\left(3, \frac{\pi}{2}\right)}$. I hope it is correct. I was careful when solving this problem. I used the given information and f

In [56]:
def find_majority_answer(answers: list) -> str:
    canonical_groups = defaultdict(int)
    canonical_to_original = {}

    print("\n--- Raw Completions ---")
    for i, answer in enumerate(answers):
        # 확실하게 text만 추출
        try:
            answer_text = answer.outputs[0].text  # <-- 핵심 수정
        except Exception:
            answer_text = str(answer)

        print(f"[{i}] {answer_text}")
        canonical = get_canonical_form(answer_text)
        canonical_groups[canonical] += 1

        if canonical not in canonical_to_original:
            canonical_to_original[canonical] = answer_text

    print("\n--- Canonical Groups ---")
    for canonical, count in canonical_groups.items():
        print(f"{canonical}: {count}")

    max_count = max(canonical_groups.values())
    for canonical, count in canonical_groups.items():
        if count == max_count:
            print(f"\n>>> Selected: {canonical_to_original[canonical]}")
            return canonical_to_original[canonical]
        
sample = dataset[0]["problem"]
prompt = f"{config['prompt'].strip()}\n\n### Problem:\n{sample.strip()}"
completions = llm.generate(prompt, sampling_params=sampling_params)

import re

def extract_boxed_answer(text: str) -> str:
    """
    '$\\boxed{...}$' 안의 내용을 추출한다. 없을 경우 fallback으로 전체 텍스트를 반환.
    """
    match = re.search(r"\\boxed\{(.+?)\}", text)
    if match:
        return match.group(1).strip()
    return text.strip()

from sympy import simplify
from sympy.parsing.latex import parse_latex

def is_correct(pred_str, answer_str) -> bool:
    try:
        pred_expr = extract_boxed_answer(pred_str)
        answer_expr = extract_boxed_answer(answer_str)
        lhs = simplify(parse_latex(pred_expr))
        rhs = simplify(parse_latex(answer_expr))
        return lhs == rhs
    except Exception as e:
        print(f"[WARN] Failed to parse or simplify: {e}")
        return pred_expr == answer_expr  # fallback: string equality
    
# Majority vote debug
pred = find_majority_answer(completions)

correct = 0
total = len(dataset)

for i in range(total):
    pred = dataset[i]["pred"][0]
    answer = dataset[i]["answer"]
    if is_correct(pred, answer):
        correct += 1

print(f"Accuracy: {correct}/{total} = {correct / total:.2%}")

Processed prompts: 100%|██████████| 1/1 [00:23<00:00, 23.70s/it, est. speed input: 7.00 toks/s, output: 4046.61 toks/s]


--- Raw Completions ---
[0] 

## Step 1: Recall the conversion formulas
The conversion from rectangular to polar coordinates uses the formulas $r = \sqrt{x^2 + y^2}$ for the radial coordinate and $\theta = \text{atan2}(y,x)$ for the angular coordinate.

## Step 2: Identify the rectangular coordinates
The given point is $(x,y) = (0,3)$.

## Step 3: Calculate the radial coordinate
Substitute the given values into the formula for $r$: $r = \sqrt{0^2 + 3^2} = \sqrt{9} = 3$.

## Step 4: Calculate the angular coordinate
Substitute the given values into the formula for $\theta$: $\theta = \text{atan2}(3,0) = \frac{\pi}{2}$, since the point lies on the positive y-axis.

## Step 5: Write the polar coordinates
Combine the calculated values of $r$ and $\theta$ to obtain the polar coordinates: $(r,\theta) = \left(3, \frac{\pi}{2}\right)$.

The final answer is: $\boxed{\left(3, \frac{\pi}{2}\right)}$. I hope it is correct. I was careful when solving this problem. I used the given information and f

In [57]:
pred

"\n\n## Step 1: Recall the conversion formulas\nThe conversion from rectangular to polar coordinates uses the formulas $r = \\sqrt{x^2 + y^2}$ for the radial coordinate and $\\theta = \\text{atan2}(y,x)$ for the angular coordinate.\n\n## Step 2: Identify the rectangular coordinates\nThe given point is $(x,y) = (0,3)$.\n\n## Step 3: Calculate the radial coordinate\nSubstitute the given values into the formula for $r$: $r = \\sqrt{0^2 + 3^2} = \\sqrt{9} = 3$.\n\n## Step 4: Calculate the angular coordinate\nSubstitute the given values into the formula for $\\theta$: $\\theta = \\text{atan2}(3,0) = \\frac{\\pi}{2}$, since the point lies on the positive y-axis.\n\n## Step 5: Write the polar coordinates\nCombine the calculated values of $r$ and $\\theta$ to obtain the polar coordinates: $(r,\\theta) = \\left(3, \\frac{\\pi}{2}\\right)$.\n\nThe final answer is: $\\boxed{\\left(3, \\frac{\\pi}{2}\\right)}$. I hope it is correct. I was careful when solving this problem. I used the given informa

In [36]:
print("Predicted Answer:", dataset[i]["pred"])
print("Completions:", dataset[i]["completions"])

Predicted Answer: ['RequestOutput(request_id=5, prompt=\'Solve the following math problem efficiently and clearly:\\n\\n- For simple problems (2 steps or fewer):\\nProvide a concise solution with minimal explanation.\\n\\n- For complex problems (3 steps or more):\\nUse this step-by-step format:\\n\\n## Step 1: [Concise description]\\n[Brief explanation and calculations]\\n\\n## Step 2: [Concise description]\\n[Brief explanation and calculations]\\n\\n...\\n\\nTherefore, the final answer is: $\\\\boxed{answer}$. I hope it is correct.\\n\\nWhere [answer] is just the final number or expression that solves the problem.\\n\\n### Problem:\\nThe results of a cross-country team\\\'s training run are graphed below. Which student has the greatest average speed? [asy]\\nfor ( int i = 1; i <= 7; ++i )\\n{\\n\\ndraw((i,0)--(i,6));\\n}\\n\\nfor ( int i = 1; i <= 5; ++i )\\n{\\n\\ndraw((0,i)--(8,i));\\n}\\ndraw((-0.5,0)--(8,0), linewidth(1));\\ndraw((0,-0.5)--(0,6), linewidth(1));\\nlabel("$O$", (0,0

In [23]:
for n in [1, 2, 4, 8, 16, 64, 128]:
    sampling_params = SamplingParams(
        temperature=config["temperature"],
        top_p=config["top_p"],
        max_tokens=config["max_tokens"],
        n=n,
        seed=0
    )
    start = time.time()
    completions = llm.generate("what is the capital of South Korea?", sampling_params=sampling_params)
    elapsed = time.time() - start
    print(f"n={n}, elapsed time: {elapsed:.2f} sec")

Processed prompts: 100%|██████████| 1/1 [00:15<00:00, 15.18s/it, est. speed input: 0.59 toks/s, output: 134.95 toks/s]


n=1, elapsed time: 15.18 sec


Processed prompts: 100%|██████████| 1/1 [00:15<00:00, 15.28s/it, est. speed input: 0.59 toks/s, output: 152.80 toks/s]


n=2, elapsed time: 15.28 sec


Processed prompts: 100%|██████████| 1/1 [00:15<00:00, 15.21s/it, est. speed input: 0.59 toks/s, output: 422.77 toks/s]


...

Processed prompts: 100%|██████████| 1/1 [00:36<00:00, 36.20s/it, est. speed input: 0.25 toks/s, output: 6976.83 toks/s]


n=128, elapsed time: 36.21 sec


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

WARNING 08-02 18:11:33 scheduler.py:1484] Sequence group 13 is preempted by PreemptionMode.SWAP mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1


RuntimeError: Aborted due to the lack of CPU swap space. Please increase the swap space to avoid this error.